In [ ]:
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.6 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Thu Apr 20 07:11:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-tuned-alpha-3b")

model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-tuned-alpha-3b")
model.half().cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50688, 4096)
    (layers): ModuleList(
      (0-15): 16 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=4096, out_features=50688, bias=False)
)

In [ ]:
!nvidia-smi

Thu Apr 20 07:15:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W /  70W |   7297MiB / 15360MiB |     34%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:

prompt = f"{system_prompt}<|USER|>What's your mood today?<|ASSISTANT|>"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=64,
  temperature=0.7,
  do_sample=True,
  top_p=0.95,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()])
)
print(tokenizer.decode(tokens[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
What's your mood today?I am currently working. How may I assist you today?


In [ ]:
def chat(input_text):
    
    prompt = f"{system_prompt}<|USER|>{input_text}<|ASSISTANT|>"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    tokens = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    stopping_criteria=StoppingCriteriaList([StopOnTokens()])
    )
    print(wrap_text_preserve_newlines(tokenizer.decode(tokens[0], skip_special_tokens=True)))


In [ ]:
chat("Write a note to Sam Altman saying they should open source GPT-4")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered
harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and
make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Write a note to Sam Altman saying they should open source GPT-4Dear Sam Altman,

I wanted to take a moment to thank you for your work on [project name]. Your expertise in language generation
and text-to-text technology has been invaluable to us.

Your work is truly important to the success of our project, and I am grateful for the opportunity to work with
you on this project. Your technical expertise and creativity have been a vital factor in our success and have
greatly contributed to the success of our project.

I have been excited to work w

In [ ]:
chat("Write a story about a Koala who could beat all the camelids at pool")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered
harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and
make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Write a story about a Koala who could beat all the camelids at poolOnce upon a time, in a small town nestled
in the heart of Africa, there lived a Koala named Camel. Camelids loved swimming in the pool at the local
pool, but the pool was too small and the water was murky.

One day, a group of travelers came to town to seek shelter from the rainy season. They were tired and hungry,
and they had been wandering for days without any sign of civilization.

As they trudged through the mud, they came upon a group of camels. The camels were unlike any

In [ ]:
chat("What is the capital of England?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered
harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and
make jokes.
- StableLM will refuse to participate in anything that could harm a human.
What is the capital of England?The capital of England is London.


In [ ]:
chat("What are the difference between Llamas, Alpacas and Vicunas?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered
harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and
make jokes.
- StableLM will refuse to participate in anything that could harm a human.
What are the difference between Llamas, Alpacas and Vicunas?Llamas, Alpacas, and Vicunas are all different
types of llamas, but they are all part of the same group of animals called llamas. They are known for their
soft, moist skin and hardworking nature.

Llamas are a type of llama, which is a species of mammal that is native to North America and Central Asia.
They are characterized by their large ears, which are used to keep their balance and to communicate with each
other.

Alpacas are another type of llama, which is a species of mammal tha

In [ ]:
chat("Are you a fan of the Simpsons? Tell me a bit about Homer.")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered
harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and
make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Are you a fan of the Simpsons? Tell me a bit about Homer.Of course I am a fan of the show! I am always eager
to learn more about the work of the talented and visionary writers who create the show. I am particularly fond
of the writing style and humor of Mr. Burns, the beloved voice of Springfield. Homer Simpson is indeed a very
special individual, and his positive character and simple lifestyle make him such a beloved character among
Springfield residents. I am also fond of the quirky characters of Mr. Burns, such as his love of puns and
wordp